# Politics in BYU

In [66]:
SCHOOL="byu"
SUBJECT="opinion"
START_YEAR=2011
FINAL_YEAR=2022
DATA_DIR="data"      # should be 'data'
OUTPUT_DIR="output"  # should be 'output'

In [67]:
cd /Users/anishka/Desktop/asdrp/mui/diversity-colleges/diversity-colleges/

/Users/anishka/Desktop/asdrp/mui/diversity-colleges/diversity-colleges


In [68]:
import sys
sys.path.append('src')
import ouraws
import ourgraphs
#import textutil

In [69]:
# NOTE: before loading, we need to be in the "diversity-colleges" folder

S3OBJECT_KEY = f"{DATA_DIR}/{SCHOOL}-{SUBJECT}-SNAPSHOT.parquet"
df = ouraws.getFromS3(S3OBJECT_KEY)

In [70]:
df.shape

(1528, 6)

In [71]:
df.head(2)

,title,url,body,year,month,day
18,Readers’ Forum: Why we should let guilt go,https://universe.byu.edu/2022/06/06/readers-fo...,Readers’ Forum: Why we should let guilt go\nWe...,2022,6,6
19,Opinion: Utah’s wedding culture is laughably e...,https://universe.byu.edu/2022/06/01/opinion-ut...,Opinion: Utah’s wedding culture is laughably e...,2022,6,1


In [72]:
import requests
import os

BIPARTISAN_API_KEY = os.environ.get("BIPARTISAN_API")

BIPARTISAN_URL = "https://api.thebipartisanpress.com/api/endpoints/beta/robert"

In [73]:
df2022 = df[df['year'] == 2022]

articles_list = df2022['body'].to_list()
for article in articles_list:
    payload = {"API": BIPARTISAN_API_KEY, "Text": article.encode("utf-8")}
    response = requests.post(BIPARTISAN_URL, data=payload)
    print(response.text)
    break

0.112398


In [74]:
df['year'].unique()

array([2022, 2021, 2020, 2018, 2017, 2016, 2015, 2023, 2019, 2014, 2013,
       2012, 2011])

In [81]:
# SYNCHRONOUS METHOD : TAKES A LONG TIME

results = []

for year in range(2022, 2024):
    df2 = df[df['year'] == year]
    print(f"Year: {year} ... {df2.shape[0]}")

    articles_list = df2['body'].to_list()
    value_sum = 0.0
    article_count = 0
    for article in articles_list:
        payload = {"API": BIPARTISAN_API_KEY, "Text": article.encode("utf-8")}
        response = requests.post(BIPARTISAN_URL, data=payload)
        try:
            value_sum += float(response.text)
            article_count += 1
            # print(f"{article_count}", end=".")
        except:
            pass  # this is a non-number
        
    results.append({
        'year' : year,
        'article_count' : article_count,
        'polarity_sum' : value_sum,
        'polarity_avg' : float(value_sum) / article_count
    })

    print(f"{year}\t{article_count}\t{value_sum}\t{value_sum/article_count}")
# for each year, get all articles' bodies & send to bipartisan press API 
#    and sum up the values & output the average for each year

Year: 2022 ... 60
2022	60	139.31874499999998	2.321979083333333
Year: 2023 ... 32
2023	32	-5.290269	-0.16532090625


In [79]:
import pandas as pd

RESULTS_FILE = f"{DATA_DIR}/{SCHOOL}-POLARITY.parquet"

results_df = pd.DataFrame.from_records(results)
results_df.to_parquet(RESULTS_FILE)